<a href="https://colab.research.google.com/github/AbiSyaibah/Data-Science/blob/main/World%20Cup%20Predict/WorldCup_Prediction_(Data_Preparation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
import pickle
import numpy as np

#01

In [2]:
all_tables=pd.read_html('https://id.wikipedia.org/wiki/Piala_Dunia_FIFA_2022#Babak_grup')

In [3]:
all_tables[14]
all_tables[33]
all_tables[33+(33-14)]
all_tables[33+2*(33-14)]
all_tables[33+3*(33-14)]
all_tables[33+4*(33-14)]
all_tables[33+5*(33-14)]
all_tables[33+6*(33-14)]

,Pos,Timlbs,Main,M,S,K,MG,KG,SG,Poin,Kualifikasi
0,1,Portugal,3,2,0,1,6,4,+2,6,Lolos ke babak gugur
1,2,Korea Selatan,3,1,1,1,4,4,0,4,Lolos ke babak gugur
2,3,Uruguay,3,1,1,1,2,2,0,4,NaN
3,4,Ghana,3,1,0,2,5,7,−2,3,NaN


In [4]:
alphabet=string.ascii_uppercase
print(alphabet)

ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [5]:
dict_table={}

for letter,i in zip(alphabet, range(14,(33+6*(33-14)),(33-14))):
  df=all_tables[i]
  df.rename(columns={df.columns[1]:'Team'},inplace=True)
  df.pop('Kualifikasi')
  dict_table[f'Group {letter}']=df

In [6]:
dict_table.keys()
dict_table['Group G']

,Pos,Team,Main,M,S,K,MG,KG,SG,Poin
0,1,Brasil,3,2,0,1,3,1,+2,6
1,2,Swiss,3,2,0,1,4,3,+1,6
2,3,Kamerun,3,1,1,1,4,4,0,4
3,4,Serbia,3,0,1,2,5,8,−3,1


In [7]:
with open('dict_table','wb') as output:
  pickle.dump(dict_table,output)

#02

In [8]:
from bs4 import BeautifulSoup
import requests

In [9]:
#Tahun piala dunia
years=[2018,2014,2010,2006,2002,1998,1994,1990,1986,1982,1978,1974,1970,1966,1962,1958,1954,1950,1938,1934,1930]


In [10]:
def get_matches(year):
    #link situs tempat data berada
    web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    #menggunakan requests untuk melakukan HTTP GET request ke alamat web yang diberikan pada variable 'web'
    response=requests.get(web)

    #menggunakan response.text untuk mengambil konten dari response HTTP tersebut
    content=response.text

    #menggunakan BeautifulSoup untuk memproses konten HTML tersebut
    soup=BeautifulSoup(content, 'lxml')

    #mengambil box tempat pertandingan dan skor berada dari file lxml
    matches=soup.find_all('div', class_='footballbox')

    #list tempat menyimpan data pertandingan
    home=[]
    score=[]
    away=[]

    #memasukkan semua data pada mathes yang diinginkan
    for match in matches:
        home.append(match.find('th',class_='fhome').get_text())
        score.append(match.find('th',class_='fscore').get_text())
        away.append(match.find('th',class_='faway').get_text())

    #memasukkan data ke dalam dictionary supaya bisa dijadikan dataframe
    football_data={'home':home,'score':score,'away':away}

    #membuat dataframe
    df_football=pd.DataFrame(football_data)
    #menambah kolom tahun piala dunia
    df_football['year']=year

    return df_football

In [11]:
def get_matches_2022(year):
    #link situs tempat data berada
    web='https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup'

    #menggunakan requests untuk melakukan HTTP GET request ke alamat web yang diberikan pada variable 'web'
    response=requests.get(web)

    #menggunakan response.text untuk mengambil konten dari response HTTP tersebut
    content=response.text

    #menggunakan BeautifulSoup untuk memproses konten HTML tersebut
    soup=BeautifulSoup(content, 'lxml')

    #mengambil box tempat pertandingan dan skor berada dari file lxml
    matches=soup.find_all('div', class_='footballbox')

    #list tempat menyimpan data pertandingan
    home=[]
    score=[]
    away=[]

    #memasukkan semua data pada mathes yang diinginkan
    for match in matches:
        home.append(match.find('th',class_='fhome').get_text())
        score.append(match.find('th',class_='fscore').get_text())
        away.append(match.find('th',class_='faway').get_text())

    #memasukkan data ke dalam dictionary supaya bisa dijadikan dataframe
    football_data={'home':home,'score':score,'away':away}

    #membuat dataframe
    df_football=pd.DataFrame(football_data)
    #menambah kolom tahun piala dunia
    df_football['year']=year

    return df_football

In [12]:
#mendapatkan semua data untuk setiap tahun piala dunia dilaksanakan
#for loop menggunakan list comprehension
fifa=[get_matches(year) for year in years]  #trypedata list

In [13]:
#join all the table in fifa into a new final dataframe
df_fifa=pd.concat(fifa,ignore_index=True)
print(df_fifa)

               home score            away  year
0           Russia    5–0    Saudi Arabia  2018
1            Egypt    0–1         Uruguay  2018
2           Russia    3–1           Egypt  2018
3          Uruguay    1–0    Saudi Arabia  2018
4          Uruguay    3–0          Russia  2018
..              ...   ...             ...   ...
800  United States    3–0        Paraguay  1930
801       Paraguay    1–0         Belgium  1930
802      Argentina    6–1   United States  1930
803        Uruguay    6–1      Yugoslavia  1930
804        Uruguay    4–2       Argentina  1930

[805 rows x 4 columns]


In [14]:
#mengimport data ke csv
df_fifa.to_csv('data_piala_dunia.csv', index=False) #index=false berguna untuk menghilangkan index dari dataframe saat dieksport

In [15]:
#membuat fixture
df_fixture=get_matches_2022(2022)

#menimport fisture ke csv
df_fixture.to_csv('data_fixture_2022.csv',index=False)

#03

In [36]:
df_historical_data=pd.read_csv('/content/data_piala_dunia.csv')
df_missing_data=pd.read_csv('/content/fifa_worldcup_missing_data.csv')
df_fixture=pd.read_csv('/content/data_fixture_2022.csv')

In [37]:
df_historical_data

,home,score,away,year
0,Russia,5–0,Saudi Arabia,2018
1,Egypt,0–1,Uruguay,2018
2,Russia,3–1,Egypt,2018
3,Uruguay,1–0,Saudi Arabia,2018
4,Uruguay,3–0,Russia,2018
...,...,...,...,...
800,United States,3–0,Paraguay,1930
801,Paraguay,1–0,Belgium,1930
802,Argentina,6–1,United States,1930
803,Uruguay,6–1,Yugoslavia,1930


1. Cleaning df_fixture

In [38]:
df_fixture['home']=df_fixture['home'].str.strip()
df_fixture['away']=df_fixture['away'].str.strip()

2. Cleaning df_missing_data

In [39]:
#melihat null data
df_missing_data[df_missing_data['year'].isnull()]

,home,score,away,year


3. Menggabungkan missing data dengan historical data

In [40]:
#menggabungkan missing data dengan historical data
df_final_data=pd.concat([df_historical_data,df_missing_data], ignore_index=True)

#memperbaiki data sehingga hanya bersisa string saja(tidak ada spasi)
df_final_data['home']=df_final_data['home'].str.strip()
df_final_data['away']=df_final_data['away'].str.strip()

In [41]:
#menghilangkan duplicate data
df_final_data.drop_duplicates(inplace=True)

#mengurutkan data
df_final_data.sort_values('year',inplace=True)

df_final_data

,home,score,away,year
804,Uruguay,4–2,Argentina,1930
787,France,4–1,Mexico,1930
788,Argentina,1–0,France,1930
789,Chile,3–0,Mexico,1930
790,Chile,1–0,France,1930
...,...,...,...,...
36,Belgium,3–0,Panama,2018
35,Mexico,0–3,Sweden,2018
34,South Korea,2–0,Germany,2018
32,South Korea,1–2,Mexico,2018


In [42]:
#Check apakah sudah tidak ada lagi data duplikat
result=df_final_data.loc[df_final_data['home']=="South Korea"]
print(result)

            home         score           away  year
492  South Korea           2–3          Italy  1986
491  South Korea           1–1       Bulgaria  1986
463  South Korea           1–3          Spain  1990
465  South Korea           0–1        Uruguay  1990
399  South Korea           0–0        Bolivia  1994
344  South Korea           1–3         Mexico  1998
274  South Korea           2–0         Poland  2002
276  South Korea           1–1  United States  2002
318  South Korea           2–3         Turkey  2002
311  South Korea  2–1 (a.e.t.)          Italy  2002
228  South Korea           2–1           Togo  2006
134  South Korea           2–0         Greece  2010
110  South Korea           0–1        Belgium  2014
109  South Korea           2–4        Algeria  2014
34   South Korea           2–0        Germany  2018
32   South Korea           1–2         Mexico  2018


4. Cleaning final data

In [45]:
#memilih semua data dayng skornya mengandung (aet)
df_final_data[df_final_data["score"].str.contains('[^\d–]')]

,home,score,away,year
776,Austria,3–2 (a.e.t.),France,1934
786,Italy,2–1 (a.e.t.),Czechoslovakia,1934
779,Italy,1–1 (a.e.t.),Spain,1934
757,Brazil,6–5 (a.e.t.),Poland,1938
756,Italy,2–1 (a.e.t.),Norway,1938
...,...,...,...,...
61,Croatia,2–1 (a.e.t.),England,2018
59,Russia,2–2 (a.e.t.),Croatia,2018
55,Colombia,1–1 (a.e.t.),England,2018
51,Croatia,1–1 (a.e.t.),Denmark,2018


In [46]:
#mengganti setiap aet dengan ""
df_final_data['score']=df_final_data["score"].str.replace("[^\d–]","",regex=True)

In [47]:
#membersihkan kolom home dan away dari spasi
df_final_data['home']=df_final_data['home'].str.strip()
df_final_data['away']=df_final_data['away'].str.strip()

In [48]:
#splitting kolom skor menjadi home dan away, lalu menambahkannya ke dataframe kita
df_final_data[["Home Goal", "Away Goal"]]=df_final_data['score'].str.split("–",expand=True)

In [49]:
#menghapus kolom score
df_final_data.drop('score', axis=1, inplace=True)

In [50]:
#merubah nama kolom biar lebih rapi
df_final_data.rename(columns={'home':'Team Tuan Rumah', 
                              'away':'Team Tamu',
                              'year':'Tahun Piala Dunia'},inplace=True)

In [51]:
#ternyata datatype goal bukan int, berarti harus kita ubah 
df_final_data.dtypes

Team Tuan Rumah      object
Team Tamu            object
Tahun Piala Dunia     int64
Home Goal            object
Away Goal            object
dtype: object

In [52]:
#membersihkan data dari spasi
df_final_data['Home Goal']=df_final_data['Home Goal'].str.strip("w/o[]a")
df_final_data['Away Goal']=df_final_data['Away Goal'].str.strip("w/o[]a")

In [53]:
#meng-drop setiap data yang tidak ada atau null
df_final_data.dropna(axis=0,inplace=True)

,Team Tuan Rumah,Team Tamu,Tahun Piala Dunia,Home Goal,Away Goal


In [54]:
#mengubah datatype nya jadi int
df_final_data=df_final_data.astype({"Home Goal":int,"Away Goal":int})

In [56]:
#membuat kolom total goal
df_final_data["Total Goal"]=df_final_data['Home Goal']+df_final_data['Away Goal']

In [57]:
df_final_data

,Team Tuan Rumah,Team Tamu,Tahun Piala Dunia,Home Goal,Away Goal,Total Goal
804,Uruguay,Argentina,1930,4,2,6
787,France,Mexico,1930,4,1,5
788,Argentina,France,1930,1,0,1
789,Chile,Mexico,1930,3,0,3
790,Chile,France,1930,1,0,1
...,...,...,...,...,...,...
36,Belgium,Panama,2018,3,0,3
35,Mexico,Sweden,2018,0,3,3
34,South Korea,Germany,2018,2,0,2
32,South Korea,Mexico,2018,1,2,3


#04 Export cleaning DataFrame

In [58]:
df_final_data.to_csv('fifa_worldcup_clean_data.csv', index=False)
df_fixture.to_csv('fifa_worldcup_clean_fixture.csv', index=False)